# Detecting Fraud on Ethereum

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import data from data/transactions.csv
transactions_df = pd.read_csv('data/transactions.csv')

# show summary statistics
transactions_df.describe()